In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
from huggingface_hub import login

from utils.helpers import find_root_dir

import numpy as np
import seaborn as sns

In [ ]:
ARDT_DIR = find_root_dir()
VERSION_OF_COMBO = "v1"  # PLEASE CHANGE TO AVOID OVERRIDING!!!!!
ENVIRONMENT_NAME = "halfcheetah"

In [ ]:
total_episodes = 1000
episode_len = 1000
total_steps = total_episodes * episode_len  # ideally

In [ ]:
# dataset_names = ["ppo_eval_halfcheetah", "arrl_nrmdp_train_halfcheetah"]
# weight_combs = []
# for i in [0.0, 0.1, 0.35, 0.65, 0.9, 1.0]:
#     weight_combs.append([round(1.0-i, 2), i])

# weight_combs

In [ ]:
dataset_names = ["ppo_test_halfcheetah_v3", "arrl_nrmdp_train_halfcheetah", "ppo_randadv_test_halfcheetah_v3_filtered"]
weight_combs = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0.5, 0.5],
    [0.5, 0.25, 0.25],
    [0.5, 0.5, 0],
    [1/3, 1/3, 1/3],
    [1/3, 2/3, 0],
    [0.1, 0.45, 0.45],
    [0.1, 0.9, 0]
]

In [ ]:
def compute_sum(ds):
    return {'returns': sum(ds['rewards'])}

for i, weights in enumerate(weight_combs):
    assert np.sum(weights) == 1, f"Weights {weights} need to sum to 1."
    episodes_per_ds = [int(w * total_episodes) for w in weights]
    j = 0
    while(sum(episodes_per_ds) != 1000):
        episodes_per_ds[j] += 1
        j += 1
        if j == len(dataset_names):
            j = 0
    
    super_dataset = None
    for dataset_name, eps in zip(dataset_names, episodes_per_ds):
        if eps == 0:
            continue
        # load dataset
        ds = load_from_disk(f"{ARDT_DIR}/datasets/{dataset_name}")
        ds = ds.map(compute_sum)
        # select last episodes_per_ds episodes
        len_ds = len(ds['observations'])
        ds = ds.select(range(len_ds-eps, len_ds))
        # # visualise selection of dataset
        # returns = (np.array(ds['rewards'])).sum(axis=1)
        # sns.displot(returns, kind="kde", bw_adjust=0.5);
        # add to super dataset
        if super_dataset is None:
            super_dataset = ds
        else:
            ds = ds.cast(super_dataset.features)
            super_dataset = concatenate_datasets([super_dataset, ds])
    # visualise combined dataset
    ds_to_vis = super_dataset.map(compute_sum)
    print(f"Datasets: {dataset_names} \n Weights: {weights}")
    sns.displot(ds_to_vis['returns'], kind="kde", bw_adjust=0.5);
    # save
    super_dataset.save_to_disk(f'{ARDT_DIR}/datasets-to-push/dataset_combo_train_{ENVIRONMENT_NAME}_v{i+1}_new')